# Lab 1: Building the Brain

## From Scratch to Framework

In this lab, you will:

1. **Part 1:** Build a "raw" ReAct agent from scratch using a simple `while` loop and text parsing
2. **Part 2:** Compare your raw agent with the project's `ReactAgent` that uses native function calling

### Learning Goals
- Understand that an agent is just a **loop** with state and reasoning
- Experience the **pain** of parsing free-text tool calls
- Appreciate why **native function calling** is used in production

### Prerequisites
- `uv pip install litellm python-dotenv`
- A valid API key in your `.env` file (e.g., `OPENAI_API_KEY=sk-...`)

---
## Setup

In [ ]:
import os
import re
import json
from dotenv import load_dotenv
from litellm import completion

load_dotenv()

MODEL = os.getenv("MODEL_NAME", "gpt-4o")
print(f"Using model: {MODEL}")

---
## Part 1: The "Raw" ReAct Agent

We'll build a ReAct agent that uses **text-based** tool calling. The LLM outputs plain text in a specific format, and we parse it to extract tool calls.

### Step 1: Define Mock Tools

First, let's create some simple tools our agent can use. These are just Python functions — no API keys needed.

In [ ]:
# Tool registry — maps tool names to functions
TOOLS = {
    "search": search,
    "calculate": calculate,
}

# Define tools as OpenAI-compatible schemas for native calling
TOOLS_SCHEMA = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Search for information on a topic. Returns relevant text.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Evaluate a mathematical expression. Returns the result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The math expression to evaluate (e.g., '15 * 500 / 100')"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]



### Step 2: The ReAct System Prompt

Define a system prompt that instructs the LLM to follow the Thought -> Action -> Observation format **exactly**.

In [ ]:
REACT_SYSTEM_PROMPT = """You are a helpful research assistant that solves tasks step by step.

You have access to these tools:
- search(query): Search for information. Input: a search query string.
- calculate(expression): Evaluate a math expression. Input: a math expression string.

Follow this EXACT format for EVERY step:

Thought: <your reasoning about what to do next>
Action: <tool_name>("<argument>")

After receiving an Observation, continue with the next Thought.

When you have enough information to answer, use:

Thought: I have enough information to answer.
Final Answer: <your complete answer>

IMPORTANT:
- ALWAYS start with a Thought before any Action.
- Use EXACTLY one Action per step.
- Wait for the Observation before your next Thought.
- Never fabricate Observations — only use real tool results.
"""

print("System prompt loaded.")

### Step 3: Build the Parser

**TODO:** Complete the `parse_action` function to extract the tool name and argument from the LLM's response.

In [ ]:
def parse_response(text: str) -> dict:
    """
    Parse the LLM's text response to extract a Final Answer.
    Native tool calls are handled separately in the agent loop.
    
    Returns:
        {"type": "final_answer", "content": "..."}
        or
        {"type": "none"}
    """
    if not text:
        return {"type": "none"}

    # Check for Final Answer
    final_match = re.search(r'Final Answer:\s*(.+)', text, re.DOTALL)
    if final_match:
        return {"type": "final_answer", "content": final_match.group(1).strip()}
    
    return {"type": "none"}



### Step 4: Build the Agent Loop

**TODO:** Complete the agent loop by:
1. Calling the LLM with `completion()`
2. Executing the tool when an action is parsed
3. Appending the observation back to messages

In [ ]:
def run_react_agent(query: str, max_steps: int = 5) -> dict:
    """
    Run a ReAct agent loop using native tool calling.
    
    Returns:
        {"answer": str, "steps": list, "total_steps": int}
    """
    messages = [
        {"role": "system", "content": REACT_SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]
    steps = []
    
    for step in range(max_steps):
        print(f"\n{'='*50}")
        print(f"Step {step + 1}")
        print(f"{'='*50}")
        
        # Call the LLM using litellm's completion() with native tools
        response = completion(
            model=MODEL, 
            messages=messages, 
            tools=TOOLS_SCHEMA,
            tool_choice="auto",
            max_tokens=512
        )
        
        message = response.choices[0].message
        assistant_text = message.content or ""
        tool_calls = message.tool_calls
        
        print(f"LLM Output:\n{assistant_text}")
        if tool_calls:
            for tc in tool_calls:
                 print(f"Tool Call: {tc.function.name}(\"{tc.function.arguments}\")")
        
        # Append assistant response (the whole message object for tool_calls) to messages
        messages.append(message)
        
        # Parse for final answer in text
        parsed = parse_response(assistant_text)
        steps.append({"step": step + 1, "raw_output": assistant_text, "tool_calls": tool_calls, "parsed": parsed})
        
        if parsed["type"] == "final_answer":
            print(f"\nFinal Answer: {parsed['content']}")
            return {"answer": parsed["content"], "steps": steps, "total_steps": step + 1}
        
        # Handle native tool calls
        if tool_calls:
            for tc in tool_calls:
                tool_name = tc.function.name
                arguments = json.loads(tc.function.arguments)
                
                # Execute the tool
                if tool_name in TOOLS:
                    # Use the first argument if it's a simple string, or pass all kwargs
                    # The schemas define 'query' or 'expression'
                    arg_value = list(arguments.values())[0] if arguments else ""
                    observation = TOOLS[tool_name](arg_value)
                else:
                    observation = f"Error: Tool {tool_name} not found."
                
                print(f"\nObservation: {observation}")
                
                # Append the tool result back to messages
                messages.append({
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "name": tool_name,
                    "content": observation
                })
        
        elif not assistant_text and not tool_calls:
            # Should not happen with good models, but handle as safety
            break
            
    return {
        "answer": assistant_text or "[Agent reached max steps without a final answer]",
        "steps": steps,
        "total_steps": max_steps,
    }



### Step 5: Test Your Agent

Try these queries — they require multi-step reasoning:

In [ ]:
# Test 1: Multi-step factual question
result = run_react_agent("What is the population of the capital of France?")
print(f"\n{'='*50}")
print(f"Answer: {result['answer']}")
print(f"Total steps: {result['total_steps']}")

In [ ]:
# Test 2: Requires search + calculation
result = run_react_agent("How tall is the Eiffel Tower in feet? What is that height divided by 3?")
print(f"\n{'='*50}")
print(f"Answer: {result['answer']}")
print(f"Total steps: {result['total_steps']}")

### Reflection: The Pain Points

After running the agent, think about these questions:

1. **Did the parser always work?** Did the LLM deviate from the expected format?
2. **How fragile is the regex?** What happens if the model writes `Action: search('query')` instead of `search("query")`?
3. **How would you handle multiple tool calls per step?** (Hint: you can't with text parsing — but native calling supports it)
4. **How would you debug a failure?** You have the raw text, but no structured trace.

---
## Part 2: Native Function Calling

Now examine the **native** approach. The same agent loop — but the LLM returns structured JSON instead of text you parse.

In [ ]:
# TOOLS_SCHEMA was moved up for use in Part 1.
print("Using TOOLS_SCHEMA defined in earlier Step.")



In [ ]:
def run_native_agent(query: str, max_steps: int = 5) -> dict:
    """
    Run an agent using native function calling.
    No text parsing needed — the API returns structured tool calls.
    """
    messages = [
        {"role": "system", "content": "You are a helpful research assistant. Use the provided tools to answer questions."},
        {"role": "user", "content": query},
    ]
    steps = []
    
    for step in range(max_steps):
        print(f"\n--- Step {step + 1} ---")
        
        # Call LLM with tools parameter
        response = completion(
            model=MODEL,
            messages=messages,
            tools=TOOLS_SCHEMA,
            tool_choice="auto",
            max_tokens=512,
        )
        
        message = response.choices[0].message
        assistant_content = message.content
        tool_calls = message.tool_calls
        
        # Add assistant message to history
        messages.append(message)
        
        step_info = {"step": step + 1, "content": assistant_content, "tool_calls": []}
        
        if assistant_content:
            print(f"Content: {assistant_content[:200]}")
        
        # Handle tool calls — structured JSON, no parsing needed!
        if tool_calls:
            for tc in tool_calls:
                func_name = tc.function.name
                func_args = json.loads(tc.function.arguments)
                
                print(f"Tool Call: {func_name}({func_args})")
                
                # Execute tool
                if func_name in TOOLS:
                    result = TOOLS[func_name](**func_args)
                else:
                    result = f"Error: Unknown tool '{func_name}'"
                
                print(f"Result: {result}")
                
                step_info["tool_calls"].append({
                    "tool": func_name, "args": func_args, "result": result
                })
                
                # Feed result back — structured format
                messages.append({
                    "tool_call_id": tc.id,
                    "role": "tool",
                    "name": func_name,
                    "content": result,
                })
        
        steps.append(step_info)
        
        # If no tool calls and we have content, the agent is done
        if not tool_calls and assistant_content:
            return {"answer": assistant_content, "steps": steps, "total_steps": step + 1}
    
    return {
        "answer": "[Max steps reached]",
        "steps": steps,
        "total_steps": max_steps,
    }

In [ ]:
# Test the native agent with the same query
result = run_native_agent("What is the population of the capital of France?")
print(f"\n{'='*50}")
print(f"Answer: {result['answer']}")
print(f"Total steps: {result['total_steps']}")

---
## Part 3: Compare and Reflect

### Side-by-Side Comparison

Run both agents on the same query and compare:

| Metric | Text-Based | Native |
|--------|-----------|--------|
| Steps taken | ? | ? |
| Parse errors | ? | 0 (guaranteed) |
| Code complexity | High (regex) | Low (structured) |
| Debugging ease | Hard | Easy |

### Next Steps

Open the project's `src/agent/react_agent.py` and compare with your native agent above. Note how it:
- Uses a `ToolRegistry` for dynamic tool management
- Implements proper error handling with try/except
- Logs each step for debugging (preview of Session 3's tracing)